In [ ]:
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.metrics import pairwise_distances
import random
from gensim.models.doc2vec import Doc2Vec
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()

In [ ]:
articles = pd.read_csv('dataset/data_articles_test.csv', delimiter=',', keep_default_na=False)
tables = pd.read_csv('dataset/dataset_distinct_tables.csv', delimiter=',', keep_default_na=False)

In [ ]:
embedding_model = Doc2Vec.load('pre_trained_models/doc2vec.bin')

In [ ]:
tables_vector = []
tables_ids = []

for i, row in tqdm(tables.iterrows()):
    
    table_title = row['table_page_title']
    
    vector_words = tknzr.tokenize(table_title)
    word_embedding = embedding_model.infer_vector(vector_words)
    
    tables_ids.append(row['table_id'])
    tables_vector.append(word_embedding) 

In [ ]:
articles_ids = []
articles_vector = []

for i, row in tqdm(articles.iterrows()):
    
    article_page_title = row['page_title']
         
    #embedding
    vector_words = tknzr.tokenize(article_page_title)
    word_embedding = embedding_model.infer_vector(vector_words)
    
    articles_ids.append(row['page_id'])
    articles_vector.append(word_embedding)

In [ ]:
def getAccuracy(idRankedTables, idQueryGoal):

    accuracy = 0

    for idTable in idRankedTables:
        
        if idTable[0] == idQueryGoal:
    
            accuracy = 1
            break;

    return accuracy

In [ ]:
def saveAccuracy(k,accuracy):
    
    if k == 1:
            
        AverageTop1.append(accuracy)
    
    if k == 5:
            
        AverageTop5.append(accuracy)
        
    if k == 10:
            
        AverageTop10.append(accuracy)
        
    if k == 20:
            
        AverageTop20.append(accuracy)

In [ ]:
AverageTop1 = []
AverageTop5 = []
AverageTop10 = []
AverageTop20 = []

topK = [1,5,10,20]

for i in tqdm(range(len(articles_vector))):
    
    idQueryGoal = articles_ids[i]
    
    distance_vector = pairwise_distances(articles_vector[i].reshape(1,50), tables_vector, metric='cosine')
    
    #creating the dataframe
    all_tables_score = []
    
    for j in range(len(tables_ids)):
        
        table_id = tables_ids[j]
        table_score = distance_vector[0][j]
        
        new_row = {"table_id": table_id,"table_score": table_score}
        
        all_tables_score.append(new_row)
        
    df_all_tables_scores = pd.DataFrame(all_tables_score)
    df_tables_sorting = df_all_tables_scores.sort_values('table_score')
    
    #compute the accuracy
    for accuracyK in topK:
        
        selected_top = df_tables_sorting.head(accuracyK)
        min_score = selected_top['table_score'].max()
        draw_tables_socres = df_tables_sorting[df_tables_sorting['table_score'] <= min_score]
        final_ranked_tables = draw_tables_socres.iloc[:,0:1].values
        
        accuracy_value = getAccuracy(final_ranked_tables,idQueryGoal)
        
        #save the accuracy on the list
        saveAccuracy(accuracyK,accuracy_value)

In [ ]:
print(str(round(np.mean(AverageTop1),4)))
print(str(round(np.mean(AverageTop5),4)))
print(str(round(np.mean(AverageTop10),4)))
print(str(round(np.mean(AverageTop20),4)))